In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
import os

import eikon as ek
ek.set_app_key('341ab68907a740f38e7b07084d4055e3982b5bdf')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50) 
pd.set_option('display.max_columns', 50) 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
df = pd.read_excel(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\111522_YahooAndBing_9125_complete_withoutInvestmentAcq.xlsx")

# set permid from float to int
for permid_column in [col for col in df.columns if 'permid' in col or "code" in col]:
    df[permid_column] = df[permid_column].astype("Int64") # convert to string later on for eikon

df[:3]
df.shape

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,date_of_financials,isin,best_match_to_ciq_isin,permid,nation,best_match_to_ciq_nation,trbc_industry_group_code,trbc_industry_group_name,total_assets,net_income,multiple_deal_involvment,full_name,full_name_legal_addon,full_name_cleaned,best_match_name,best_match_to_ciq_name,best_match_review_amount,best_match_cosine,best_match_link,best_match_type,link_hannes,link_hannes_nonstrict,link_marius,firm_hannes,firm_hannes_nonstrict,firm_marius,review_amount_hannes,review_amount_hannes_nonstrict,review_amount_marius,link_found_hannes,link_found_hannes_nonstrict,similarity_hannes,similarity_hannes_nonstrict,similarity_marius,similarity_mean,marius_equalto_hannes
0,34391733382,2008-01-07,2008-03-04,1,2.829622e+08,0.0,100.0,2007-12-31,CA67072Q1046,CA67072Q1046,4295861226,Canada,Canada,501020,Oil & Gas,6.856333e+08,2.642212e+07,0,NuVista Energy Ltd,Ltd,NuVista Energy,NuVista Energy,NuVista Energy Ltd.,8.0,1.00,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,hannes,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,NaN,NuVista Energy,NuVista Energy,NaN,8.0,8.0,NaN,original review link,original review link,1.0,1.00,0.0,0.666667,0
1,34391733368,2008-01-07,2008-03-05,1,4.397050e+07,0.0,100.0,2007-12-31,US4519061018,NaN,4295904213,United States,New Zealand,521020,"Machinery, Tools, Heavy Vehicles, Trains & Ships",1.408352e+09,1.766600e+07,1,L-1 Identity Solutions Inc,Inc,L-1 Identity Solutions,Identity Solutions,Identity Solutions Laboratories Limited,NaN,0.83,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,nonstrict,NaN,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,NaN,NaN,Identity Solutions,NaN,NaN,NaN,NaN,no link found,transformed overview link,0.0,0.83,0.0,0.276667,1
2,34391297298,2008-01-08,2008-07-18,1,5.716800e+07,0.0,100.0,2007-12-31,US62845B1044,NaN,4295914259,United States,United States,551010,Banking Services,9.573430e+08,4.226000e+06,0,MutualFirst Financial Inc,Inc,MutualFirst Financial,Mutual of Omaha,Mutual of Omaha Insurance Company,805.0,0.36,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,nonstrict,NaN,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,NaN,NaN,Mutual of Omaha,NaN,NaN,805.0,NaN,no link found,transformed overview link,0.0,0.36,0.0,0.120000,1


(9124, 43)

# get static firm infos

## get industry trbc code

In [ ]:
ids = df["permid"].astype(str).tolist()
fields = ['TR.ISIN', 'TR.TRBCIndustryCode'] # check the data item browser

df_list = []  # Will be a list of dataframes
batchsize = 500

for i in range(0, len(ids), batchsize):
    batch = ids[i:i+batchsize]
    df_trbc, err = ek.get_data(batch, fields)
    df_list.append(df_trbc)
    print(f"{i}:{i+batchsize} / {len(ids)} done")
df_trbc = pd.concat(df_list).reset_index(drop=True).rename(columns={"Instrument": "permid", "ISIN": "isin", "TRBC Industry Code": "trbc_industry_group_code"})

df_trbc = df_trbc.drop_duplicates(subset=["permid"], keep='first') # duplicates due to serial acquirers

df_trbc

## merge

In [16]:
df = df.merge(df_trbc, how="left", on="permid", suffixes=("_old", "_new"))

df["isin"] = np.where(df["isin_old"].isna() == True, df["isin_new"], df["isin_old"])
df["trbc_industry_group_code"] = np.where(df["trbc_industry_group_code_old"].isna() == True, df["trbc_industry_group_code_new"], df["trbc_industry_group_code_old"])

df = df.drop(columns=["isin_old", "isin_new", "trbc_industry_group_code_old", "trbc_industry_group_code_new"])

df["ciq_isin_match"] = np.where(df["isin"] == df["best_match_to_ciq_isin"], 1, 0)

df.insert(df.columns.get_loc("date_of_financials")+1, "isin", df.pop("isin"))
df.insert(df.columns.get_loc("best_match_to_ciq_nation")+1, "trbc_industry_group_code", df.pop("trbc_industry_group_code"))
df.insert(df.columns.get_loc("best_match_to_ciq_isin")+1, "ciq_isin_match", df.pop("ciq_isin_match"))

df[:3]
df.shape

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,date_of_financials,isin,best_match_to_ciq_isin,ciq_isin_match,permid,nation,best_match_to_ciq_nation,trbc_industry_group_code,trbc_industry_group_name,total_assets,net_income,multiple_deal_involvment,full_name,full_name_legal_addon,full_name_cleaned,best_match_name,best_match_to_ciq_name,best_match_review_amount,best_match_cosine,best_match_link,best_match_type,link_hannes,link_hannes_nonstrict,link_marius,firm_hannes,firm_hannes_nonstrict,firm_marius,review_amount_hannes,review_amount_hannes_nonstrict,review_amount_marius,link_found_hannes,link_found_hannes_nonstrict,similarity_hannes,similarity_hannes_nonstrict,similarity_marius,similarity_mean,marius_equalto_hannes
0,34391733382,2008-01-07,2008-03-04,1,2.829622e+08,0.0,100.0,2007-12-31,CA67072Q1046,CA67072Q1046,1,4295861226,Canada,Canada,501020,Oil & Gas,6.856333e+08,2.642212e+07,0,NuVista Energy Ltd,Ltd,NuVista Energy,NuVista Energy,NuVista Energy Ltd.,8.0,1.00,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,hannes,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,NaN,NuVista Energy,NuVista Energy,NaN,8.0,8.0,NaN,original review link,original review link,1.0,1.00,0.0,0.666667,0
1,34391733368,2008-01-07,2008-03-05,1,4.397050e+07,0.0,100.0,2007-12-31,US4519061018,NaN,0,4295904213,United States,New Zealand,521020,"Machinery, Tools, Heavy Vehicles, Trains & Ships",1.408352e+09,1.766600e+07,1,L-1 Identity Solutions Inc,Inc,L-1 Identity Solutions,Identity Solutions,Identity Solutions Laboratories Limited,NaN,0.83,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,nonstrict,NaN,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,NaN,NaN,Identity Solutions,NaN,NaN,NaN,NaN,no link found,transformed overview link,0.0,0.83,0.0,0.276667,1
2,34391297298,2008-01-08,2008-07-18,1,5.716800e+07,0.0,100.0,2007-12-31,US62845B1044,NaN,0,4295914259,United States,United States,551010,Banking Services,9.573430e+08,4.226000e+06,0,MutualFirst Financial Inc,Inc,MutualFirst Financial,Mutual of Omaha,Mutual of Omaha Insurance Company,805.0,0.36,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,nonstrict,NaN,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,NaN,NaN,Mutual of Omaha,NaN,NaN,805.0,NaN,no link found,transformed overview link,0.0,0.36,0.0,0.120000,1


(9124, 44)

In [4]:
ids = ["4295911976", "CA5658511020"] #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.RIC', 'TR.OrganizationID', 'TR.UltimateParentId', 'TR.CompanyName', 'TR.FormerlyKnownAsName', 'TR.SucceededBy', 'TR.TRBCIndustryCode'] # check the data item browser

df_trbc, err = ek.get_data(ids, fields)
df_trbc

,Instrument,ISIN,RIC,Organization PermID,Ultimate Parent Id,Company Name,Formerly Known As Name,Organization Succeeded By,TRBC Industry Code
0,4295911976,US92343V1044,VZ.N,4295911976,4295911976,Verizon Communications Inc,Bell Atlantic Corp,,57401010
1,CA5658511020,CA5658511020,MAR.TO^L10,4295861595,4295861595,Marathon PGM Corp,,Stillwater Canada Inc,51201010


In [10]:
df.to_excel(Path.joinpath(Path(r'W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample'), f'{time.strftime("%m%d%y")}_9124_firms.xlsx'), index=False)

# generate high_quality proxies

In [32]:
df = pd.read_excel(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\111822_9124_firms.xlsx")

df[:3]
df.shape

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,date_of_financials,isin,best_match_to_ciq_isin,ciq_isin_match,permid,nation,best_match_to_ciq_nation,trbc_industry_group_code,trbc_industry_group_name,total_assets,net_income,multiple_deal_involvment,full_name,full_name_legal_addon,full_name_cleaned,best_match_name,best_match_to_ciq_name,best_match_review_amount,best_match_cosine,best_match_link,best_match_type,link_hannes,link_hannes_nonstrict,link_marius,firm_hannes,firm_hannes_nonstrict,firm_marius,review_amount_hannes,review_amount_hannes_nonstrict,review_amount_marius,link_found_hannes,link_found_hannes_nonstrict,similarity_hannes,similarity_hannes_nonstrict,similarity_marius,similarity_mean,marius_equalto_hannes
0,34391733382,2008-01-07,2008-03-04,1,2.829622e+08,0.0,100.0,2007-12-31,CA67072Q1046,CA67072Q1046,1,4.295861e+09,Canada,Canada,501020.0,Oil & Gas,6.856333e+08,2.642212e+07,0,NuVista Energy Ltd,Ltd,NuVista Energy,NuVista Energy,NuVista Energy Ltd.,8.0,1.00,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,hannes,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,NaN,NuVista Energy,NuVista Energy,NaN,8.0,8.0,NaN,original review link,original review link,1.0,1.00,0.0,0.666667,0
1,34391733368,2008-01-07,2008-03-05,1,4.397050e+07,0.0,100.0,2007-12-31,US4519061018,NaN,0,4.295904e+09,United States,New Zealand,521020.0,"Machinery, Tools, Heavy Vehicles, Trains & Ships",1.408352e+09,1.766600e+07,1,L-1 Identity Solutions Inc,Inc,L-1 Identity Solutions,Identity Solutions,Identity Solutions Laboratories Limited,NaN,0.83,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,nonstrict,NaN,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,NaN,NaN,Identity Solutions,NaN,NaN,NaN,NaN,no link found,transformed overview link,0.0,0.83,0.0,0.276667,1
2,34391297298,2008-01-08,2008-07-18,1,5.716800e+07,0.0,100.0,2007-12-31,US62845B1044,NaN,0,4.295914e+09,United States,United States,551010.0,Banking Services,9.573430e+08,4.226000e+06,0,MutualFirst Financial Inc,Inc,MutualFirst Financial,Mutual of Omaha,Mutual of Omaha Insurance Company,805.0,0.36,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,nonstrict,NaN,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,NaN,NaN,Mutual of Omaha,NaN,NaN,805.0,NaN,no link found,transformed overview link,0.0,0.36,0.0,0.120000,1


(9124, 44)

In [39]:
df1 = df.copy()

# trbc to int
df1["trbc_industry_group_code"] = df1["trbc_industry_group_code"].astype("Int64")

# insert year
df1["year"] = df1['date_announced'].dt.to_period('Y')
df1.insert(df1.columns.get_loc("eikon_deal_id")+1, "year", df1.pop("year"))

# helper to sort after isin / trbc not na
df1["isin_true"] = df1["isin"].isna()
df1["trbc_industry_group_code_true"] = df1["trbc_industry_group_code"].isna()

df1 = df1.sort_values(by=["acquiror", "isin_true", "trbc_industry_group_code_true", "ciq_isin_match", "year", "best_match_cosine"], ascending=[0, 1, 1, 0, 0, 0]).reset_index(drop=True)

# add id
df1["id"] = range(1, df1.shape[0]+1)
df1.insert(0, "id", df1.pop("id"))

df1 = df1.drop(columns=["isin_true", "trbc_industry_group_code_true"])
df1 = df1.replace(pd.NA, np.nan)

df1.head(3)
df1.shape

,id,eikon_deal_id,year,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,date_of_financials,isin,best_match_to_ciq_isin,ciq_isin_match,permid,nation,best_match_to_ciq_nation,trbc_industry_group_code,trbc_industry_group_name,total_assets,net_income,multiple_deal_involvment,full_name,full_name_legal_addon,full_name_cleaned,best_match_name,best_match_to_ciq_name,best_match_review_amount,best_match_cosine,best_match_link,best_match_type,link_hannes,link_hannes_nonstrict,link_marius,firm_hannes,firm_hannes_nonstrict,firm_marius,review_amount_hannes,review_amount_hannes_nonstrict,review_amount_marius,link_found_hannes,link_found_hannes_nonstrict,similarity_hannes,similarity_hannes_nonstrict,similarity_marius,similarity_mean,marius_equalto_hannes
0,1,154085312869,2021,2021-01-04,2022-01-04,1,2.487641e+09,0.0,100.0,2020-12-31,US15135B1017,US15135B1017,1,4.295902e+09,United States,United States,561020,Healthcare Providers & Services,6.871900e+10,1.808000e+09,1,Centene Corp,Corp,Centene,Centene,Centene Corporation,3800.0,1.0,https://www.glassdoor.com/Reviews/Centene-Reviews-E13542.htm,hannes,https://www.glassdoor.com/Reviews/Centene-Reviews-E13542.htm,https://www.glassdoor.com/Reviews/Centene-Reviews-E13542.htm,https://www.glassdoor.co.uk/Reviews/Centene-Reviews-E13542.htm,Centene,Centene,Centene,3800.0,3600.0,2703.0,original review link,transformed overview link,1.0,1.0,1.00,1.000000,1
1,2,154087026971,2021,2021-01-05,2021-02-02,1,2.243156e+08,0.0,100.0,2020-12-31,CA0084741085,CA0084741085,1,4.295861e+09,Canada,Canada,512010,Metals & Mining,9.614679e+09,5.382160e+08,1,Agnico Eagle Mines Ltd,Ltd,Agnico Eagle Mines,Agnico Eagle Mines,Agnico Eagle Mines Limited,18.0,1.0,https://www.glassdoor.com/Reviews/Agnico-Eagle-Mines-Reviews-E2700590.htm,hannes,https://www.glassdoor.com/Reviews/Agnico-Eagle-Mines-Reviews-E2700590.htm,https://www.glassdoor.com/Reviews/Agnico-Eagle-Mines-Reviews-E2700590.htm,https://www.glassdoor.co.uk/Reviews/Agnico-Eagle-Reviews-E6853.htm,Agnico Eagle Mines,Agnico Eagle Mines,Agnico Eagle,18.0,18.0,41.0,transformed overview link,transformed overview link,1.0,1.0,0.86,0.953333,0
2,3,154087025317,2021,2021-01-07,2021-06-21,1,1.845752e+09,0.0,100.0,2020-12-31,US62886E1082,US62886E1082,1,4.295905e+09,United States,United States,572010,Software & IT Services,7.449000e+09,-7.900000e+07,1,NCR Corp,Corp,NCR,NCR,NCR Corporation,5800.0,1.0,https://www.glassdoor.com/Reviews/NCR-Reviews-E4212.htm,hannes,https://www.glassdoor.com/Reviews/NCR-Reviews-E4212.htm,https://www.glassdoor.com/Reviews/NCR-Reviews-E4212.htm,https://www.glassdoor.com/Reviews/NCR-Reviews-E4212.htm,NCR,NCR,NCR,5800.0,5800.0,4820.0,transformed overview link,transformed overview link,1.0,1.0,1.00,1.000000,1


(9124, 46)

In [38]:
df1.to_excel(Path.joinpath(Path(r'W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample'), f'{time.strftime("%m%d%y")}_9124_sorted.xlsx'), index=False)

### get alternative firm names

In [ ]:
# create target_permid list
# ids = ["4295911976", "CA5658511020"] #first convert to int (without decimal), then to str, since get_data needs strings in the list
# fields = ['TR.ISIN', 'TR.RIC', 'TR.OrganizationID', 'TR.UltimateParentId', 'TR.CompanyName', 'TR.FormerlyKnownAsName', 'TR.SucceededBy', 'TR.TRBCIndustryCode'] # check the data item browser

ids = df["permid"].astype(str).tolist()
fields = ['TR.ISIN', 'TR.TRBCIndustryCode'] # check the data item browser

df_list = []  # Will be a list of dataframes
batchsize = 500

for i in range(0, len(ids), batchsize):
    batch = ids[i:i+batchsize]
    df_trbc, err = ek.get_data(batch, fields)
    df_list.append(df_trbc)
    print(f"{i}:{i+batchsize} / {len(ids)} done")
df_trbc = pd.concat(df_list).reset_index(drop=True).rename(columns={"Instrument": "permid", "ISIN": "isin", "TRBC Industry Code": "trbc_industry_group_code"})

df_trbc = df_trbc.drop_duplicates(subset=["permid"], keep='first') # duplicates due to serial acquirers

df_trbc

### create list of "formerly known as"

In [ ]:
df_formerly = df_static.groupby('Instrument')['Formerly Known As Name'].apply(list).reset_index(name='other_names')
df_formerly

,Instrument,other_names
0,4295899948,[Google Inc]
1,4295904162,"[Heritage Propane Partners L P, Energy Transfe..."
2,4295911990,"[Chemex-Delaware Inc, Chemex Pharmaceuticals I..."


#  merge from firm level to deal level

In [19]:
df_similarity = df.copy()
# df_similarity['date_announced'] = pd.to_datetime(df_similarity['date_announced'], format="%d/%m/%Y") # not needed if xlsx with date columns is imported

# get acq df
df_similarity_acq = df_similarity[(df_similarity["acquiror"] == 1)]

new_names = [(i,'acquiror_'+i) for i in df_similarity_acq.iloc[:, df_similarity_acq.columns.get_loc("shares_owned_after_transaction")+1:].columns.values] # prefix
df_similarity_acq = df_similarity_acq.rename(columns = dict(new_names))

# df_similarity_acq.head()
df_similarity_acq.shape

# get target df
df_similarity_target = df_similarity[(df_similarity["acquiror"] == 0)] #grab only targets

new_names = [(i,'target_'+i) for i in df_similarity_target.iloc[:, df_similarity_target.columns.get_loc("shares_owned_after_transaction")+1:].columns.values] # prefix
df_similarity_target = df_similarity_target.rename(columns = dict(new_names))

df_similarity_target = df_similarity_target.drop(labels=df_similarity_target.iloc[:, 1:df_similarity_target.columns.get_loc("shares_owned_after_transaction")+1].columns.values, axis=1) # taken from acq df, since identical

# df_similarity_target.head()
df_similarity_target.shape

# merge
df_merge = df_similarity_acq.merge(df_similarity_target, how="left", on="eikon_deal_id")

df_merge.head(3)
df_merge.shape

(4562, 44)

(4562, 38)

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,acquiror_date_of_financials,acquiror_isin,acquiror_best_match_to_ciq_isin,acquiror_ciq_isin_match,acquiror_permid,acquiror_nation,acquiror_best_match_to_ciq_nation,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,acquiror_total_assets,acquiror_net_income,acquiror_multiple_deal_involvment,acquiror_full_name,acquiror_full_name_legal_addon,acquiror_full_name_cleaned,acquiror_best_match_name,acquiror_best_match_to_ciq_name,acquiror_best_match_review_amount,...,target_full_name,target_full_name_legal_addon,target_full_name_cleaned,target_best_match_name,target_best_match_to_ciq_name,target_best_match_review_amount,target_best_match_cosine,target_best_match_link,target_best_match_type,target_link_hannes,target_link_hannes_nonstrict,target_link_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_firm_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_review_amount_marius,target_link_found_hannes,target_link_found_hannes_nonstrict,target_similarity_hannes,target_similarity_hannes_nonstrict,target_similarity_marius,target_similarity_mean,target_marius_equalto_hannes
0,34391733382,2008-01-07,2008-03-04,1,2.829622e+08,0.0,100.0,2007-12-31,CA67072Q1046,CA67072Q1046,1,4.295861e+09,Canada,Canada,501020.0,Oil & Gas,6.856333e+08,2.642212e+07,0,NuVista Energy Ltd,Ltd,NuVista Energy,NuVista Energy,NuVista Energy Ltd.,8.0,...,Rider Resources Ltd,Ltd,Rider Resources,Rider Resources,Rider Resources Inc.,NaN,1.0,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,hannes,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,Rider Resources,Rider Resources,Rider Resources,NaN,NaN,1.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.000000,1
1,34391733368,2008-01-07,2008-03-05,1,4.397050e+07,0.0,100.0,2007-12-31,US4519061018,NaN,0,4.295904e+09,United States,New Zealand,521020.0,"Machinery, Tools, Heavy Vehicles, Trains & Ships",1.408352e+09,1.766600e+07,1,L-1 Identity Solutions Inc,Inc,L-1 Identity Solutions,Identity Solutions,Identity Solutions Laboratories Limited,NaN,...,Bioscrypt Inc,Inc,Bioscrypt,Bioscrypt,"Bioscrypt, Inc.",2.0,1.0,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,hannes,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.ca/Reviews/Bioscrypt-Reviews-E24042.htm,Bioscrypt,Bioscrypt,Bioscrypt,2.0,2.0,2.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.000000,1
2,34391297298,2008-01-08,2008-07-18,1,5.716800e+07,0.0,100.0,2007-12-31,US62845B1044,NaN,0,4.295914e+09,United States,United States,551010.0,Banking Services,9.573430e+08,4.226000e+06,0,MutualFirst Financial Inc,Inc,MutualFirst Financial,Mutual of Omaha,Mutual of Omaha Insurance Company,805.0,...,MFB Corp,Corp,MFB,MFB,My Food Bag Group Limited,3.0,1.0,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,nonstrict,"https://www.glassdoor.com/job-listing/occupational-therapist-mary-free-bed-rehabilitation-hospital-JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,"occupational therapist mary free bed rehabilitation hospital JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",MFB,MFB,NaN,3.0,3.0,wrong link,transformed overview link,0.0,1.0,1.0,0.666667,1


(4562, 81)

In [ ]:
df_merge.to_excel(Path.joinpath(Path(r'W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample'), f'{time.strftime("%m%d%y")}_4562_deals.xlsx'), index=False)

In [19]:
df_merge[df_merge["eikon_deal_id"] == 154083651144]

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,acquiror_date_of_financials,acquiror_isin,acquiror_best_match_to_ciq_isin,acquiror_ciq_isin_match,acquiror_permid,acquiror_nation,acquiror_best_match_to_ciq_nation,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,acquiror_total_assets,acquiror_net_income,acquiror_multiple_deal_involvment,acquiror_full_name,acquiror_full_name_legal_addon,acquiror_full_name_cleaned,acquiror_best_match_name,acquiror_best_match_to_ciq_name,acquiror_best_match_review_amount,...,target_full_name,target_full_name_legal_addon,target_full_name_cleaned,target_best_match_name,target_best_match_to_ciq_name,target_best_match_review_amount,target_best_match_cosine,target_best_match_link,target_best_match_type,target_link_hannes,target_link_hannes_nonstrict,target_link_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_firm_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_review_amount_marius,target_link_found_hannes,target_link_found_hannes_nonstrict,target_similarity_hannes,target_similarity_hannes_nonstrict,target_similarity_marius,target_similarity_mean,target_marius_equalto_hannes
1774,154083651144,2017-11-28,2018-02-28,1,3.627865e+09,0.0,100.0,2016-12-31,GB00B15FWH70,NaN,0,4295898981,United Kingdom,United Kingdom,533010,Hotels & Entertainment Services,1.599600e+09,1.011546e+08,0,Cineworld Group PLC,PLC,Cineworld Group,Cineworld Cinemas,Cineworld Cinemas Ltd,780.0,...,Regal Entertainment Group,NaN,Regal Entertainment Group,Regal Regal entertainment group,NaN,NaN,0.91,"https://www.glassdoor.com/Reviews/Regal-Regal-entertainment-group-Reviews-EI_IE14917.0,5_KO6,31.htm",nonstrict,https://www.glassdoor.com/Reviews/Regal-Reviews-E14917.htm,"https://www.glassdoor.com/Reviews/Regal-Regal-entertainment-group-Reviews-EI_IE14917.0,5_KO6,31.htm",NaN,Regal,Regal Regal entertainment group,NaN,3500.0,NaN,NaN,transformed overview link,transformed jobs link,0.45,0.91,0.0,0.453333,0


#  check how many suited deals

## set filter criteria

In [20]:
# reviews = [i for i in range(30,60,10)]
reviews = [50]
# years = [i for i in range(2008, 2013, 1)]
years = [2012]
cosine_tresholds = list(map(lambda x: x/10.0, range(5, 11, 1))) #range doesnt work with floats

for review in reviews:
    for yearr in years:
        for cosine_treshold in cosine_tresholds:
            df_filtered = df_merge[
                (df_merge["target_best_match_review_amount"] >= review) &
                (df_merge["acquiror_best_match_review_amount"] >= review) &
                ((df_merge["target_best_match_cosine"] >= cosine_treshold) | df_merge["target_ciq_isin_match"] == 1) &
                ((df_merge["acquiror_best_match_cosine"] >= cosine_treshold) | df_merge["acquiror_ciq_isin_match"] == 1) &
                (df_merge["date_announced"].dt.year >= yearr) &
                (df_merge["acquiror_isin"].isna() == False) &
                (df_merge["target_isin"].isna() == False) &
                (df_merge["acquiror_trbc_industry_group_code"].isna() == False)
                # (df["acquiror_multiple_deal_involvment"] == 0)
                ]

            print(review, cosine_treshold, yearr,":",df_filtered.shape[0])

50 0.5 2012 : 246
50 0.6 2012 : 229
50 0.7 2012 : 217
50 0.8 2012 : 204
50 0.9 2012 : 194
50 1.0 2012 : 187


only non-serial acquiror with at least 50 reviews starting 2012 (2008) & 1.0 cosine = 276 (434)  
deals with at least 50 reviews starting 2012 & 1.0 (0.7) cosine = 127 (180) 

In [21]:
df_merge_filtered = df_merge[
    (df_merge["target_best_match_review_amount"] >= 50) &
    (df_merge["acquiror_best_match_review_amount"] >= 50) &
    (df_merge["target_best_match_cosine"] >= 0.9) &
    (df_merge["acquiror_best_match_cosine"] >= 0.5) &
    (df_merge["date_announced"].dt.year >= 2012) &
    (df_merge["acquiror_isin"].isna() == False) &
    (df_merge["target_isin"].isna() == False) &
    (df_merge["acquiror_trbc_industry_group_code"].isna() == False)
    # (df_merge["acquiror_multiple_deal_involvment"] == 0)
    ]

df_merge_filtered[:3]
df_merge_filtered.shape

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,acquiror_date_of_financials,acquiror_isin,acquiror_best_match_to_ciq_isin,acquiror_ciq_isin_match,acquiror_permid,acquiror_nation,acquiror_best_match_to_ciq_nation,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,acquiror_total_assets,acquiror_net_income,acquiror_multiple_deal_involvment,acquiror_full_name,acquiror_full_name_legal_addon,acquiror_full_name_cleaned,acquiror_best_match_name,acquiror_best_match_to_ciq_name,acquiror_best_match_review_amount,...,target_full_name,target_full_name_legal_addon,target_full_name_cleaned,target_best_match_name,target_best_match_to_ciq_name,target_best_match_review_amount,target_best_match_cosine,target_best_match_link,target_best_match_type,target_link_hannes,target_link_hannes_nonstrict,target_link_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_firm_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_review_amount_marius,target_link_found_hannes,target_link_found_hannes_nonstrict,target_similarity_hannes,target_similarity_hannes_nonstrict,target_similarity_marius,target_similarity_mean,target_marius_equalto_hannes
732,34392674380,2012-04-10,2012-05-15,1,6.140480e+08,0.0,96.448,2012-03-30,US2358511028,US2358511028,1,4.295904e+09,United States,United States,561010.0,Healthcare Equipment & Supplies,3.042096e+10,2.060483e+09,1,Danaher Corp,Corp,Danaher,Danaher,Danaher Corporation,596.0,...,X-Rite Inc,Inc,X-Rite,X Rite,"X-Rite, Incorporated",68.0,1.0,https://www.glassdoor.com/Reviews/X-Rite-Reviews-E2048.htm,hannes,https://www.glassdoor.com/Reviews/X-Rite-Reviews-E2048.htm,https://www.glassdoor.com/Reviews/X-Rite-Reviews-E2048.htm,NaN,X Rite,X Rite,NaN,68.0,68.0,NaN,transformed overview link,transformed overview link,1.0,1.0,0.0,0.666667,0
742,34392679945,2012-04-30,2012-10-05,1,5.260675e+09,0.0,100.000,2012-03-31,US29273R1095,US29273V1008,0,4.295904e+09,United States,United States,501030.0,Oil & Gas Related Equipment and Services,1.761991e+10,1.077408e+09,1,Energy Transfer Partners LP,LP,Energy Transfer Partners,Energy Transfer,Energy Transfer LP,439.0,...,Sunoco Inc,Inc,Sunoco,Sunoco,Sunoco LP,455.0,1.0,https://www.glassdoor.com/Reviews/Sunoco-Reviews-E629.htm,hannes,https://www.glassdoor.com/Reviews/Sunoco-Reviews-E629.htm,https://www.glassdoor.com/Reviews/Sunoco-Reviews-E629.htm,NaN,Sunoco,Sunoco,NaN,455.0,432.0,NaN,original review link,transformed overview link,1.0,1.0,0.0,0.666667,0
743,34392044153,2012-04-30,2012-08-01,1,3.918852e+09,0.0,100.000,2012-03-24,US4364401012,US4364401012,1,4.295907e+09,United States,United States,561010.0,Healthcare Equipment & Supplies,6.048256e+09,4.430400e+07,1,Hologic Inc,Inc,Hologic,Hologic,"Hologic, Inc.",878.0,...,Gen-Probe Inc,Inc,Gen-Probe,Gen Probe,Gen-Probe Incorporated,56.0,1.0,https://www.glassdoor.com/Reviews/Gen-Probe-Reviews-E15134.htm,hannes,https://www.glassdoor.com/Reviews/Gen-Probe-Reviews-E15134.htm,https://www.glassdoor.com/Reviews/Gen-Probe-Reviews-E15134.htm,https://www.glassdoor.co.uk/Reviews/Gen-Probe-Reviews-E15134.htm,Gen Probe,Gen Probe,Gen Probe,56.0,56.0,57.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.000000,1


(179, 81)